In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

bookDF=pd.read_csv('datasets/books/books.csv')
bookDF=bookDF.drop(['image_url','small_image_url','title','best_book_id','isbn','isbn13'],axis=1)
ratingsDF = pd.read_csv('datasets/books/ratings.csv')

# Any results you write to the current directory are saved as output.

In [2]:
bookDF

,id,book_id,work_id,books_count,authors,original_publication_year,original_title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
0,1,2767052,2792775,272,Suzanne Collins,2008.0,The Hunger Games,eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317
1,2,3,4640799,491,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543
2,3,41865,3212258,226,Stephenie Meyer,2005.0,Twilight,en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439
3,4,2657,3275794,487,Harper Lee,1960.0,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267
4,5,4671,245494,1356,F. Scott Fitzgerald,1925.0,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,7130616,7392860,19,Ilona Andrews,2010.0,Bayou Moon,eng,4.09,17204,18856,1180,105,575,3538,7860,6778
9996,9997,208324,1084709,19,Robert A. Caro,1990.0,Means of Ascent,eng,4.25,12582,12952,395,303,551,1737,3389,6972
9997,9998,77431,2393986,60,Patrick O'Brian,1977.0,The Mauritius Command,eng,4.35,9421,10733,374,11,111,1191,4240,5180
9998,9999,8565083,13433613,7,Peggy Orenstein,2011.0,Cinderella Ate My Daughter: Dispatches from th...,eng,3.65,11279,11994,1988,275,1002,3765,4577,2375


In [3]:
ratingsDF

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4
...,...,...,...
981751,10000,48386,5
981752,10000,49007,4
981753,10000,49383,5
981754,10000,50124,5


In [4]:
ratingsDF.drop_duplicates(keep='last', inplace=True) 

In [5]:
ratingsDF['user_id'].nunique()

53424

In [6]:
ratingsDF.shape

(980112, 3)

This part of the code is responsible for building the item-item similarity matrix
We first convert build a list of dictionary. Each dictionary corresponds to a single book. The user_id is the key, while rating given by the user for the book is its value.

In [7]:
listOfDictonaries=[]
indexMap = {}
reverseIndexMap = {}
ptr=0;
testdf = ratingsDF
testdf=testdf[['user_id','rating']].groupby(testdf['book_id'])
for groupKey in testdf.groups.keys():
    tempDict={}

    groupDF = testdf.get_group(groupKey)
    for i in range(0,len(groupDF)):
        tempDict[groupDF.iloc[i,0]]=groupDF.iloc[i,1]
    indexMap[ptr]=groupKey
    reverseIndexMap[groupKey] = ptr
    ptr=ptr+1
    listOfDictonaries.append(tempDict)

In [8]:
listOfDictonaries

[{314: 5,
  439: 3,
  588: 5,
  1169: 4,
  1185: 4,
  2077: 4,
  2487: 4,
  2900: 5,
  3662: 4,
  3922: 5,
  5379: 5,
  5461: 3,
  5885: 5,
  6630: 5,
  7563: 3,
  9246: 1,
  10140: 4,
  10146: 5,
  10246: 4,
  10335: 4,
  10610: 5,
  10944: 5,
  11854: 4,
  11927: 4,
  12471: 5,
  13282: 5,
  13544: 5,
  15494: 5,
  16377: 4,
  16913: 5,
  17434: 5,
  17663: 5,
  17984: 5,
  18031: 5,
  18313: 5,
  18361: 4,
  20076: 3,
  20467: 3,
  20848: 5,
  21228: 5,
  21487: 4,
  21713: 5,
  22602: 4,
  23576: 4,
  23612: 4,
  24326: 5,
  24389: 4,
  24499: 5,
  24834: 4,
  24845: 5,
  25164: 4,
  25182: 5,
  25214: 4,
  26145: 4,
  26629: 4,
  26661: 4,
  28158: 4,
  28767: 5,
  29123: 3,
  29703: 5,
  30681: 5,
  31001: 4,
  32055: 4,
  32305: 5,
  32592: 4,
  32635: 5,
  32748: 5,
  32923: 5,
  33065: 4,
  33697: 4,
  33716: 5,
  33872: 5,
  33890: 3,
  37284: 5,
  37834: 5,
  38080: 4,
  38082: 5,
  38475: 4,
  39423: 3,
  41074: 3,
  42404: 5,
  43985: 4,
  44243: 2,
  44397: 5,
  45269: 4,

In [9]:
indexMap

{0: 1,
 1: 2,
 2: 3,
 3: 4,
 4: 5,
 5: 6,
 6: 7,
 7: 8,
 8: 9,
 9: 10,
 10: 11,
 11: 12,
 12: 13,
 13: 14,
 14: 15,
 15: 16,
 16: 17,
 17: 18,
 18: 19,
 19: 20,
 20: 21,
 21: 22,
 22: 23,
 23: 24,
 24: 25,
 25: 26,
 26: 27,
 27: 28,
 28: 29,
 29: 30,
 30: 31,
 31: 32,
 32: 33,
 33: 34,
 34: 35,
 35: 36,
 36: 37,
 37: 38,
 38: 39,
 39: 40,
 40: 41,
 41: 42,
 42: 43,
 43: 44,
 44: 45,
 45: 46,
 46: 47,
 47: 48,
 48: 49,
 49: 50,
 50: 51,
 51: 52,
 52: 53,
 53: 54,
 54: 55,
 55: 56,
 56: 57,
 57: 58,
 58: 59,
 59: 60,
 60: 61,
 61: 62,
 62: 63,
 63: 64,
 64: 65,
 65: 66,
 66: 67,
 67: 68,
 68: 69,
 69: 70,
 70: 71,
 71: 72,
 72: 73,
 73: 74,
 74: 75,
 75: 76,
 76: 77,
 77: 78,
 78: 79,
 79: 80,
 80: 81,
 81: 82,
 82: 83,
 83: 84,
 84: 85,
 85: 86,
 86: 87,
 87: 88,
 88: 89,
 89: 90,
 90: 91,
 91: 92,
 92: 93,
 93: 94,
 94: 95,
 95: 96,
 96: 97,
 97: 98,
 98: 99,
 99: 100,
 100: 101,
 101: 102,
 102: 103,
 103: 104,
 104: 105,
 105: 106,
 106: 107,
 107: 108,
 108: 109,
 109: 110,
 110: 11

In [10]:
reverseIndexMap

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 18: 17,
 19: 18,
 20: 19,
 21: 20,
 22: 21,
 23: 22,
 24: 23,
 25: 24,
 26: 25,
 27: 26,
 28: 27,
 29: 28,
 30: 29,
 31: 30,
 32: 31,
 33: 32,
 34: 33,
 35: 34,
 36: 35,
 37: 36,
 38: 37,
 39: 38,
 40: 39,
 41: 40,
 42: 41,
 43: 42,
 44: 43,
 45: 44,
 46: 45,
 47: 46,
 48: 47,
 49: 48,
 50: 49,
 51: 50,
 52: 51,
 53: 52,
 54: 53,
 55: 54,
 56: 55,
 57: 56,
 58: 57,
 59: 58,
 60: 59,
 61: 60,
 62: 61,
 63: 62,
 64: 63,
 65: 64,
 66: 65,
 67: 66,
 68: 67,
 69: 68,
 70: 69,
 71: 70,
 72: 71,
 73: 72,
 74: 73,
 75: 74,
 76: 75,
 77: 76,
 78: 77,
 79: 78,
 80: 79,
 81: 80,
 82: 81,
 83: 82,
 84: 83,
 85: 84,
 86: 85,
 87: 86,
 88: 87,
 89: 88,
 90: 89,
 91: 90,
 92: 91,
 93: 92,
 94: 93,
 95: 94,
 96: 95,
 97: 96,
 98: 97,
 99: 98,
 100: 99,
 101: 100,
 102: 101,
 103: 102,
 104: 103,
 105: 104,
 106: 105,
 107: 106,
 108: 107,
 109: 108,
 110: 109,
 111: 11

We then use sklearn's DictVectorizer() function to create vectors corresponding to each book. We are trying to create a vector space with users as column vectors. Each point in the vector space represents a book. Rating of the book given an user is its magnitude. We then calculate similarity/distance between books in this vector space.

In [11]:
from sklearn.feature_extraction import DictVectorizer
dictVectorizer = DictVectorizer(sparse=True)
vector = dictVectorizer.fit_transform(listOfDictonaries)

We finally use sklearn's consine_similarity function to calculate pairwise similarity matrix 

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
pairwiseSimilarity = cosine_similarity(vector)

At last we define a function that takes an input id and outputs 5 top most recommendations. Please note that this id is the column id in book.csv. Values of id are in  [1,10000]

In [13]:
def printBookDetails(bookID):
    print("Title:", bookDF[bookDF['id']==bookID]['original_title'].values[0])
    print("Author:",bookDF[bookDF['id']==bookID]['authors'].values[0])
    print("Printing Book-ID:",bookID)
    print("=================++++++++++++++=========================")


def getTopRecommandations(bookID):
    row = reverseIndexMap[bookID]
    #print("------INPUT BOOK--------")
    #printBookDetails(bookID)
    #print("-------RECOMMENDATIONS----------")
    #similarBookIDs = [printBookDetails(indexMap[i]) for i in np.argsort(pairwiseSimilarity[row])[-12:-2][::-1]]
    return np.argsort(pairwiseSimilarity[row])[-12:-2][::-1]

# Getting Base Book for each User

In [14]:
#getting base book for all users
booksUsedForRecommendations = []
usersList = ratingsDF.groupby(by='user_id')
userIds = ratingsDF['user_id'].unique()

for x in range(len(userIds)):
    allUserRatings = pd.DataFrame(usersList.get_group(userIds[x]))
    allUserRatings = allUserRatings.sort_values(['rating'], ascending=[False])
    itemUsedForRecommendation = allUserRatings.head(1)
    booksUsedForRecommendations.append(itemUsedForRecommendation)

# Recommending 10 books for each user based on base book

In [15]:
ids = []
recommendations = []
titles = []
for x in range(len(userIds)):
    userId = userIds[x]
    bookId = booksUsedForRecommendations[x].book_id.item()
    result = getTopRecommandations(bookId) #array of book ids
    for i in range (len(result)):
        ids.append(userIds[x])
        recommendations.append(result[i] + 1)
        bookTitle = bookDF[bookDF['id']==(result[i]+1)]['original_title'].values[0]
        titles.append(bookTitle)
allUsersRecommendations_df = pd.DataFrame(list(zip(ids, recommendations,titles)),
            columns =['user_id', 'book_id', 'title'])

In [16]:
booksUsedForRecommendations[0].book_id.item()

1

In [17]:
getTopRecommandations(1)

array([30,  1, 19,  2, 92,  4, 15,  8, 36, 35])

In [18]:
allUsersRecommendations_df

,user_id,book_id,title
0,314,31,The Help
1,314,2,Harry Potter and the Philosopher's Stone
2,314,20,Mockingjay
3,314,3,Twilight
4,314,93,The Secret Garden
...,...,...,...
534235,27590,6180,Paddy Clarke Ha Ha Ha
534236,27590,5478,The Line of Beauty
534237,27590,6607,The Rainbow
534238,27590,2612,A Suitable Boy


# Explaning using KNN

# Getting Similar Items

In [19]:
knn_items_dict = {}

In [20]:
min_val = pairwiseSimilarity.min() - 1

for i in range(10000):
            pairwiseSimilarity[i, i] = min_val
            knn_to_item_i = (-pairwiseSimilarity[i, :]).argsort()[:10]
            knn_items_dict[i] = knn_to_item_i

# Calculating intersection between ratings and similar items

In [21]:
pointer = 0
explanations = []
for x in range (len(userIds)):
    user_ratings = usersList.get_group(userIds[x]).book_id.values #give it userID
    counter = 0
    while counter < 10:
        #print(pointer)
        recBookId = recommendations[pointer]
        #print(recBookId)
        recBookTitle = titles[pointer]
        #print(recommendedMovie)
        #rec_Origin_id = int(recommendedMovie.origin_iid)
        #print(rec_item_id)
        sim_items = knn_items_dict[recBookId - 1]
        explanation =  set(sim_items) & set(user_ratings)
        explanations.append(explanation)
        pointer = pointer + 1
        counter = counter + 1
allUsersRecommendations_df['explanations'] = explanations

In [22]:
allUsersRecommendations_df

,user_id,book_id,title,explanations
0,314,31,The Help,"{56, 66, 44}"
1,314,2,Harry Potter and the Philosopher's Stone,"{17, 3, 12, 6}"
2,314,20,Mockingjay,"{17, 20, 22}"
3,314,3,Twilight,"{1, 35, 46}"
4,314,93,The Secret Garden,"{1, 3, 36, 41}"
...,...,...,...,...
534235,27590,6180,Paddy Clarke Ha Ha Ha,{}
534236,27590,5478,The Line of Beauty,{}
534237,27590,6607,The Rainbow,{}
534238,27590,2612,A Suitable Boy,{}


# Model Fidelity

In [23]:
expl = allUsersRecommendations_df[[len(x) > 0 for x in allUsersRecommendations_df.explanations]]
fidelity = expl.groupby('user_id')['title'].count() / 10
modelFidelity = sum(fidelity)/len(userIds)

In [24]:
modelFidelity

0.024756663671757793